## Summary

This notebook processes the list of universities in the Leiden Ranking to locate their article in the English Wikipedia and different metadata of the article.

# Packages

In [ ]:
import pandas as pd
import urllib
import csv
import re

# To query Wikipedia API
import requests
import json
from requests.packages.urllib3.exceptions import InsecureRequestWarning

# To avoid warnings
requests.packages.urllib3.disable_warnings(InsecureRequestWarning)

# Leiden Ranking

<div class="alert alert-block alert-info"> <b>More info:</b> <a href="https://www.leidenranking.com/ranking/2021/list">CWTS Leiden Ranking 2021</a> </div>

Leiden Ranking (2021) includes a total of 1225 universities. This dataset includes their short name and id (it is needed to connect each university to its Wikipedia page).

In [ ]:
df_leiden = pd.read_csv('data/leiden_ranking_2021.tsv', sep='\t')
df_leiden

## 1.2. GRID

<div class="alert alert-block alert-info"> <b>More info:</b> <a href="https://www.grid.ac/">GRID</a> </div>

GRID includes a total of 13,158 universities with a link to a Wikipedia page.

In [ ]:
df_grid = pd.read_csv('data/wikipedia_grid_univ_2021.csv', sep=';')
df_grid = df_grid[~df_grid['wikipedia_url'].isna()]
df_grid

Wikipedia URLs are cleaned, English Wikipedia links are selected and the beginning of the URL is removed to obtain the Wikipedia page title.

In [ ]:
df_grid.wikipedia_url = df_grid.wikipedia_url.str.replace('http://', '', regex=False)
df_grid.wikipedia_url = df_grid.wikipedia_url.str.replace('https://', '', regex=False)
df_grid = df_grid[df_grid.wikipedia_url.str.startswith('en.wikipedia.org/')].copy()
df_grid.wikipedia_url = df_grid.wikipedia_url.str.replace('en.wikipedia.org/wiki/', '', regex=False)

URLs are unquoted to avoid problems when joinning them. There is a total of 13,151 universities with a English Wikipedia page.

In [ ]:
df_grid.wikipedia_url = [urllib.parse.unquote(x) for x in df_grid.wikipedia_url]
df_grid

## 1.3. Intermediate table

We use an intermediate table to univocally link the Leiden Ranking with the GRID data.

In [ ]:
df_link = pd.read_csv('data/leiden_grid_2021.csv', sep=';')
df_link.wikipedia_url = df_link.wikipedia_url.str.replace('http://', '', regex=False)
df_link.wikipedia_url = df_link.wikipedia_url.str.replace('https://', '', regex=False)
df_link = df_link[df_link.wikipedia_url.str.startswith('en.wikipedia.org/')].copy()
df_link.wikipedia_url = df_link.wikipedia_url.str.replace('en.wikipedia.org/wiki/', '', regex=False)
df_link.wikipedia_url = [urllib.parse.unquote(x) for x in df_link.wikipedia_url]

As we only want GRID table to include the established year we only keep this data. Both tables are joinned by the Wikipedia title. Despite some universities are in both tables they are not joinned due to differences in this field. However, it is the most accurate way to do it.

In [ ]:
df_link_grid = pd.merge(df_link[['id', 'country_iso_num_code', 'wikipedia_url']], df_grid[['wikipedia_url', 'established_year']].drop_duplicates(), how='left', on='wikipedia_url')

It generates some duplicated institutions. They have the same Wikipedia title.

In [ ]:
df_grid[df_grid.wikipedia_url=='Istituto_Superiore_per_le_Industrie_Artistiche']

Furthermore, there are 9 universities that do not include Wikipedia URL or are not included in GRID, **as is the case with China Univ Petrol E China**.

In [ ]:
df_leiden[~df_leiden['id'].isin(df_link_grid.id)]

Before fix some errors (duplicates, missing Wikipedia pages...), these datasets are merged.

In [ ]:
df = df_leiden.merge(df_link_grid, how='left', on='id')

In [ ]:
len(df.id.drop_duplicates())

# Data cleaning

## Duplicated URLs

There are 3 pages duplicated (same Wikipedia URL) that need to be fixed manually. This is due to they have more than one established year.

In [ ]:
df[(df.wikipedia_url.duplicated()) & (~df.wikipedia_url.isna())]

In all of them, the lower date is maintained.

In [ ]:
df.loc[df.wikipedia_url=='University_of_Southampton', 'established_year'] = 1952
df.loc[df.wikipedia_url=='Carnegie_Mellon_University', 'established_year'] = 1900
df.loc[df.wikipedia_url=='Kent_State_University', 'established_year'] = 1910

In [ ]:
df = df.drop_duplicates().copy()
df.shape

## Missing Wikipedia page

There are 9 universities without Wikipedia page. They are reviewed and 5 of them are found, the rest are not in the English Wikipedia.

In [ ]:
df[df.wikipedia_url.isnull()]

In [ ]:
df.loc[df.id==9678,'wikipedia_url'] = 'China_University_of_Petroleum'
df.loc[df.id==10551,'wikipedia_url'] = 'University_of_Paris'
df.loc[df.id==10561,'wikipedia_url'] = 'Grenoble_Alpes_University'
df.loc[df.id==19006,'wikipedia_url'] = 'Clermont_Auvergne_University'
df.loc[df.id==33166,'wikipedia_url'] = 'University_of_Health_Sciences_(Turkey)'

The rest are removed from the dataset.

In [ ]:
df = df[~df.wikipedia_url.isnull()].copy()
df

## URLs to sections

There are two Wikipedia pages URLs that are hyperlinks to specific page sections, both of them are fixed.

In [ ]:
df[df.wikipedia_url.str.contains('#', regex=False)]

In [ ]:
df.loc[df.id==1371,'wikipedia_url'] = 'Higher_School_of_Economics'
df.loc[df.id==9596,'wikipedia_url'] = 'Renmin_University_of_China'

## Final check

Let's go to check if the same Wikipedia page appears twice after the previous changes.

In [ ]:
df[df.wikipedia_url.duplicated()]

There is only one case, but it is fine as this Wikipedia page includes information about the two.

In [ ]:
df[df.wikipedia_url=='China_University_of_Petroleum']

# Wikipedia data

Wikipedia pages (from [Wikipedia Knowledge Graph](https://doi.org/10.5281/zenodo.6346900)) dataset is imported and reduced to only article pages.

In [ ]:
df_wiki = pd.read_csv('data/page.tsv', sep='\t', quoting=csv.QUOTE_NONE)
df_wiki = df_wiki[df_wiki['namespace']==0].copy()
df_wiki

As some of them don't include the title they are removed.

In [ ]:
df_wiki = df_wiki[~df_wiki.title.isna()].copy()

1219 university pages are found in the Wikipedia dataset (one is duplicated).

In [ ]:
df_wiki[df_wiki.title.isin(df.wikipedia_url)]

<div class="alert alert-block alert-warning"> JiLin Agr Univ Wikipedia page was removed in 2015 (<a href="https://en.wikipedia.org/w/index.php?title=Special:Log&logid=67775234">More info</a>) </div>

However, there is one page that is not available anymore, so it is removed and the dataset is now reduced.

In [ ]:
df[~df.wikipedia_url.isin(df_wiki.title)]

In [ ]:
df = df[df.wikipedia_url.isin(df_wiki.title)].copy()
df

## Redirects

141 university pages are redirects, so we need to fix these links using the Wikipedia API.

In [ ]:
redirects = pd.DataFrame({'from':df_wiki[(df_wiki['is_redirect']==1) & (df_wiki.title.isin(df.wikipedia_url))]['page_id'],
                          'to':None})
redirects

In [ ]:
count = 0
for i in redirects['from'].tolist():
    count+=1
    print(round(100*count/redirects.shape[0], 2), end='\r')
    url = 'https://en.wikipedia.org/w/api.php?action=query&format=json&redirects&pageids=' + str(i)
    query = requests.get(url, verify=False)
    response = json.loads(query.text)
    redirects.loc[redirects['from']==i,'to'] = int(list(response['query']['pages'].keys())[0])

In [ ]:
redirects

There is a problem with one redirect.

In [ ]:
redirects[redirects['from'] == redirects['to']]

After looking for more information in Wikipedia it is found that the real page is 1714346.

In [ ]:
df_wiki[df_wiki.page_id==66560028]

In [ ]:
df_wiki[df_wiki.page_id==1714346]

In [ ]:
redirects.loc[redirects['from'] == 66560028, 'to'] = 1714346

Before solving this problem both datasets are joinned.

In [ ]:
df = df[['id', 'university', 'country_iso_num_code', 'wikipedia_url', 'established_year']].merge(df_wiki, how='inner', left_on='wikipedia_url', right_on='title')
df

### Final check

The length of Wikipedia pages that are not redirect are checked to identify more problematic ones.

In [ ]:
df[~df.page_id.isin(redirects['from'])].sort_values(by='len', ascending=True)[['wikipedia_url', 'len']].head(5)

This is the case of `China University of Geosciences`. It is a disambiguation page. The real one is `China_University_of_Geosciences_(Wuhan)`.

In [ ]:
df_wiki[df_wiki['page_id']==60119299]

This problem will be solved with the other redirects.

In [ ]:
redirects = pd.concat([redirects, pd.DataFrame(data={'from':[60119299], 'to':[2790409]})])

Redirects are fixed.

In [ ]:
for i in redirects['from'].tolist():
    df.loc[df['page_id']==i, df_wiki.columns.tolist()] = df_wiki[df_wiki['page_id']==int(redirects.loc[redirects['from']==i,'to'].values[0])].values.flatten().tolist()

Data looks fine.

In [ ]:
df.sort_values(by='len', ascending=True)[['wikipedia_url', 'len']].head(5)

In [ ]:
df.to_csv('data/base_leiden.tsv', index=False, sep='\t')

## Language links

The number of language links is retrieved by Wikipedia API as well as the URL. To do it a new variable and `data.frame` are created.

In [ ]:
df['langlinks'] = 0
langlinks = pd.DataFrame(columns={'id', 'university','lang', 'url'})

In [ ]:
count=0
for i in range(len(df)):
    count+=1
    print(round(100*count/df.shape[0],2), end='\r')
    url = 'https://en.wikipedia.org/w/api.php?action=query&prop=langlinks&format=json&lllimit=max&llprop=url&pageids=' + str(df['page_id'].tolist()[i])
    query = requests.get(url, verify=False)
    response = json.loads(query.text)
    
    # English Wikipedia (title from Wikipedia dataset is used)
    langlinks.loc[len(langlinks)] = pd.Series({'id':df['id'].tolist()[i], 'university':df['university'].tolist()[i], 'lang':'en', 'url':'https://en.wikipedia.org/wiki/'+df['title'].tolist()[i] })
    
    # Other languages
    if ('langlinks' in response['query']['pages'][str(df['page_id'].tolist()[i])]):
        df.loc[df['page_id']==df['page_id'].tolist()[i],'langlinks'] = len(response['query']['pages'][str(df['page_id'].tolist()[i])]['langlinks'])
        for i_l in response['query']['pages'][str(df['page_id'].tolist()[i])]['langlinks']:
            langlinks.loc[len(langlinks)] = pd.Series({'id':df['id'].tolist()[i], 'university':df['university'].tolist()[i], 'lang':i_l['lang'], 'url':urllib.parse.unquote(i_l['url'])})
    else:
        df.loc[df['page_id']==df['page_id'].tolist()[i],'langlinks'] = 0

In [ ]:
langlinks[langlinks.url.duplicated()]

In [ ]:
langlinks[langlinks.url=='https://mk.wikipedia.org/wiki/Универзитет_Богазичи']

In [ ]:
langlinks[~langlinks.url.isin(wikidata.url)].url.tolist()

In [ ]:
wikidata[~wikidata.url.isin(langlinks.url)]

In [ ]:
wikidata[~wikidata.url.isin(langlinks.url)].url.tolist()

In [ ]:
langlinks.to_csv('data/langlinks.tsv', sep='\t', index=False)

In [ ]:
wikidata = pd.DataFrame(columns={'url', 'watchers', 'revisions', 'minor_edits', 'editors', 'author', 'created_at', 'assessment',
                                 'characters', 'words', 'references', 'unique_references', 'sections'})

In [ ]:
count=0
for i in langlinks['url']:
    count+=1
    print(round(100*count/langlinks.shape[0],2), end='\r')
    
    try:
        url_info = 'https://xtools.wmflabs.org/api/page/articleinfo/' + re.sub('https://|/wiki', '', i)
        query_info = requests.get(url_info, verify=False)
        response_info = json.loads(query_info.text)

        url_prose = 'https://xtools.wmflabs.org/api/page/prose/' + re.sub('https://|/wiki', '', i)
        query_prose = requests.get(url_prose, verify=False)
        response_prose = json.loads(query_prose.text)

        if(response_info['assessment']):
            assessment = response_info['assessment']['value']
        else:
            assessment = None

        wikidata.loc[len(wikidata)] = pd.Series({'url':i,
                                                 'watchers':response_info['watchers'],
                                                 'revisions':response_info['revisions'],
                                                 'minor_edits':response_info['minor_edits'],
                                                 'editors':response_info['editors'],
                                                 'author':response_info['author'],
                                                 'created_at':response_info['created_at'],
                                                 'assessment':assessment,
                                                 'characters':response_prose['characters'],
                                                 'words':response_prose['words'],
                                                 'references':response_prose['references'],
                                                 'unique_references':response_prose['unique_references'],
                                                 'sections':response_prose['sections']})
    except:
        print('Error: '+ i)
    

In [ ]:
wikidata_fix = pd.read_csv('fixerrors.tsv', sep='\t')
wikidata_fix

In [ ]:
wikidata[wikidata.url.isin(wikidata_fix.url)]

In [ ]:
wikidata = pd.concat([wikidata, wikidata_fix]).drop_duplicates()
wikidata

In [ ]:
wikidata[wikidata.url.isin(langlinks.url)]

In [ ]:
langlinks[langlinks.url.isin(wikidata.url)]

In [ ]:
langlinks[~langlinks.url.isin(wikidata.url)]

In [ ]:
wikidata.to_csv('data/wikimetrics.tsv', sep='\t', index=False)

### Views

In [ ]:
url_main = 'https://wikimedia.org/api/rest_v1/metrics/pageviews/per-article/en.wikipedia/all-access/all-agents/Main/monthly/2015010100/2022070100'
query_main = requests.get(url_main, verify=False, headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64)'})
response_main = json.loads(query_main.text)

In [ ]:
df_views = pd.DataFrame({'university':wikidata.url})
df_views

In [ ]:
for x in response_main['items']:
    df_views[x['timestamp']] = None

In [ ]:
count=0
for i in wikidata['url']:
    count+=1
    print(round(100*count/wikidata.shape[0],2), end='\r')
    
    try:
        url_views = 'https://wikimedia.org/api/rest_v1/metrics/pageviews/per-article/' + re.sub('https://|\..*', '', i) + '.wikipedia/all-access/user/' + re.sub('.*org/wiki/', '', i) + '/monthly/2015010100/2022070100'
        query_views = requests.get(url_views, verify=False, headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64)'})
        response_views = json.loads(query_views.text)
        
        for x in response_views['items']:
            df_views.loc[df_views.university==i,x['timestamp']] = x['views']
        
    except:
        print('Error: '+ i)
    

In [ ]:
url_views = 'https://wikimedia.org/api/rest_v1/metrics/pageviews/per-article/ceb.wikipedia/all-access/user/Vrije_Universiteit_Brussel_%2F_Campus_Etterbeek/monthly/2015010100/2022070100'
query_views = requests.get(url_views, verify=False, headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64)'})
response_views = json.loads(query_views.text)

for x in response_views['items']:
    df_views.loc[df_views.university=='https://ceb.wikipedia.org/wiki/Vrije_Universiteit_Brussel_/_Campus_Etterbeek',x['timestamp']] = x['views']

In [ ]:
df_views.to_csv('page/views.tsv', sep='\t', index=False)